In [228]:
#Fundamental librarys to math and stats process
import numpy as np
import numpy.random as nr
import scipy.stats as ss
import math
#data prepared
import pandas as pd

#ML preprocessi
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.pipeline import make_pipeline
from sklearn import feature_selection as fs
from sklearn.model_selection import KFold

from sklearn.preprocessing import RobustScaler,Normalizer, MinMaxScaler,FunctionTransformer, PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge

# ML algorithms models
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC

# ML Evaluations
import sklearn.metrics as sklm
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split

#Ploting
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [229]:
df = (pd.read_csv('train_df_enc.csv'))
df.head()
x= df.drop(['Unnamed: 0','target', 'ward'], axis=1)
y= df['target']
x.head()

,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,dw_07,...,ADM2_PCODE,ADM2_ID,ADM2_TYPE,ADM3_PCODE,ADM3_ID,ADM3_TYPE,average_household_Size,x_cordinate,y_cordinate,z_cordinate
0,1674.45058,5888.20750,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,0.005750,0.031466,...,5,0,0,25,2,0,3.516501,-0.149468,-0.159996,0.986768
1,1736.99230,6735.33812,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,0.007575,0.123547,...,5,0,0,25,2,0,3.877587,-0.617622,-0.496455,0.747956
2,2403.57591,7273.04995,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,0.006686,0.022632,...,5,0,0,25,2,0,3.025929,-0.645915,-0.493202,0.763000
3,1740.78737,5734.49046,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,0.005255,0.068909,...,5,0,0,25,2,0,3.294194,-0.447440,-0.405587,0.890170
4,1730.51451,6657.23835,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,0.004985,0.009150,...,5,0,0,25,2,0,3.846971,-0.416870,-0.382647,0.906554


In [240]:
categorical_columns = x.columns.tolist()


categorical_features = [col for c, col in enumerate(x.columns) \
                        if not ( np.issubdtype(x.dtypes[c], np.number )  )  ]

np.issubdtype(x.dtypes['dw_00'],np.number)
x.dtypes

total_households          float64
total_individuals         float64
dw_00                     float64
dw_01                     float64
dw_02                     float64
                           ...   
ADM3_TYPE                   int64
average_household_Size    float64
x_cordinate               float64
y_cordinate               float64
z_cordinate               float64
Length: 67, dtype: object

In [233]:
print( len(categorical_features), x.shape, y.shape, y.mean()  )

0 (2822, 67) (2822,) 24.507554434444103


In [192]:
x=np.array(x)
y=np.array(y)
print(x.shape)
print(y.shape)

(2822, 67)
(2822,)


In [193]:
# splt train test
nr.seed(9988)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.15)

In [209]:
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor

lasso = make_pipeline(RobustScaler(), Lasso())
ENet = make_pipeline(RobustScaler(),ElasticNet())
KRR = KernelRidge(kernel='polynomial', degree=2)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.02070684579659932, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=500000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1, metric='rmse')

model_lgb = lgb.LGBMRegressor(boosting_type='gbdt', objective='huber',num_leaves=16,
                              learning_rate=0.02070684579659932, n_estimators=500000,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11,metric='rmse',
                              reg_alpha= 15,
                              reg_lambda= 15)



GBoost = GradientBoostingRegressor(n_estimators=500000, learning_rate=0.02070684579659932,
                                   max_depth=3, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [210]:
lasso.fit(x_train,y_train)
ENet.fit(x_train,y_train)
KRR.fit(x_train,y_train)

KernelRidge(alpha=1, coef0=1, degree=2, gamma=None, kernel='polynomial',
            kernel_params=None)

In [211]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
n_folds = 5
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(df.values)
    rmse= np.sqrt(-cross_val_score(model, x_train,y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

# Averaged base models class
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X_train,y_train):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X_train,y_train,)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X_test):
        predictions = np.column_stack([model.predict(X_test) for model in self.models_])
        return np.mean(predictions, axis=1)

In [212]:
# XGBoost:
model_xgb.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)],eval_metric='rmse',early_stopping_rounds=250)
xgb_train_pred = model_xgb.predict(x_test)
# print(rmsle(y_df, xgb_train_pred))

[0]	validation_0-rmse:25.5791	validation_1-rmse:25.7327
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 250 rounds.
[1]	validation_0-rmse:25.0798	validation_1-rmse:25.2311
[2]	validation_0-rmse:24.5858	validation_1-rmse:24.7366
[3]	validation_0-rmse:24.1014	validation_1-rmse:24.2521
[4]	validation_0-rmse:23.6257	validation_1-rmse:23.7793
[5]	validation_0-rmse:23.1593	validation_1-rmse:23.3167
[6]	validation_0-rmse:22.6997	validation_1-rmse:22.8568
[7]	validation_0-rmse:22.2534	validation_1-rmse:22.4122
[8]	validation_0-rmse:21.8182	validation_1-rmse:21.9804
[9]	validation_0-rmse:21.3916	validation_1-rmse:21.5562
[10]	validation_0-rmse:20.9738	validation_1-rmse:21.138
[11]	validation_0-rmse:20.562	validation_1-rmse:20.7265
[12]	validation_0-rmse:20.1627	validation_1-rmse:20.3285
[13]	validation_0-rmse:19.7698	validation_1-rmse:19.9367
[14]	validation_0-rmse:19.3856	validation_1-rmse:19.55

[141]	validation_0-rmse:3.70291	validation_1-rmse:4.36216
[142]	validation_0-rmse:3.68549	validation_1-rmse:4.3474
[143]	validation_0-rmse:3.67077	validation_1-rmse:4.33596
[144]	validation_0-rmse:3.65457	validation_1-rmse:4.3208
[145]	validation_0-rmse:3.64011	validation_1-rmse:4.30996
[146]	validation_0-rmse:3.62711	validation_1-rmse:4.29812
[147]	validation_0-rmse:3.61244	validation_1-rmse:4.28497
[148]	validation_0-rmse:3.60063	validation_1-rmse:4.27513
[149]	validation_0-rmse:3.58619	validation_1-rmse:4.2618
[150]	validation_0-rmse:3.57208	validation_1-rmse:4.25043
[151]	validation_0-rmse:3.5608	validation_1-rmse:4.24491
[152]	validation_0-rmse:3.54836	validation_1-rmse:4.23911
[153]	validation_0-rmse:3.53535	validation_1-rmse:4.22741
[154]	validation_0-rmse:3.52194	validation_1-rmse:4.21532
[155]	validation_0-rmse:3.51171	validation_1-rmse:4.20822
[156]	validation_0-rmse:3.50084	validation_1-rmse:4.1987
[157]	validation_0-rmse:3.4888	validation_1-rmse:4.19139
[158]	validation_0-r

[283]	validation_0-rmse:2.94649	validation_1-rmse:3.88937
[284]	validation_0-rmse:2.9446	validation_1-rmse:3.89161
[285]	validation_0-rmse:2.94181	validation_1-rmse:3.88766
[286]	validation_0-rmse:2.9389	validation_1-rmse:3.88525
[287]	validation_0-rmse:2.9364	validation_1-rmse:3.88326
[288]	validation_0-rmse:2.93462	validation_1-rmse:3.88288
[289]	validation_0-rmse:2.93313	validation_1-rmse:3.88523
[290]	validation_0-rmse:2.93055	validation_1-rmse:3.88591
[291]	validation_0-rmse:2.92942	validation_1-rmse:3.88827
[292]	validation_0-rmse:2.92764	validation_1-rmse:3.88907
[293]	validation_0-rmse:2.92578	validation_1-rmse:3.89004
[294]	validation_0-rmse:2.92318	validation_1-rmse:3.88719
[295]	validation_0-rmse:2.92006	validation_1-rmse:3.88355
[296]	validation_0-rmse:2.91806	validation_1-rmse:3.88394
[297]	validation_0-rmse:2.91589	validation_1-rmse:3.88308
[298]	validation_0-rmse:2.9147	validation_1-rmse:3.88225
[299]	validation_0-rmse:2.91274	validation_1-rmse:3.88238
[300]	validation_0

[425]	validation_0-rmse:2.67986	validation_1-rmse:3.79299
[426]	validation_0-rmse:2.67935	validation_1-rmse:3.79197
[427]	validation_0-rmse:2.67727	validation_1-rmse:3.79165
[428]	validation_0-rmse:2.67563	validation_1-rmse:3.79203
[429]	validation_0-rmse:2.67434	validation_1-rmse:3.7937
[430]	validation_0-rmse:2.67317	validation_1-rmse:3.79198
[431]	validation_0-rmse:2.67229	validation_1-rmse:3.79143
[432]	validation_0-rmse:2.67069	validation_1-rmse:3.79134
[433]	validation_0-rmse:2.66836	validation_1-rmse:3.79071
[434]	validation_0-rmse:2.66688	validation_1-rmse:3.79197
[435]	validation_0-rmse:2.66482	validation_1-rmse:3.79142
[436]	validation_0-rmse:2.66355	validation_1-rmse:3.79183
[437]	validation_0-rmse:2.66228	validation_1-rmse:3.79336
[438]	validation_0-rmse:2.66011	validation_1-rmse:3.79332
[439]	validation_0-rmse:2.65894	validation_1-rmse:3.79267
[440]	validation_0-rmse:2.65771	validation_1-rmse:3.79222
[441]	validation_0-rmse:2.65664	validation_1-rmse:3.79185
[442]	validatio

[567]	validation_0-rmse:2.47586	validation_1-rmse:3.75343
[568]	validation_0-rmse:2.47451	validation_1-rmse:3.75205
[569]	validation_0-rmse:2.47359	validation_1-rmse:3.75308
[570]	validation_0-rmse:2.47248	validation_1-rmse:3.75145
[571]	validation_0-rmse:2.4709	validation_1-rmse:3.75151
[572]	validation_0-rmse:2.47038	validation_1-rmse:3.75172
[573]	validation_0-rmse:2.46861	validation_1-rmse:3.75152
[574]	validation_0-rmse:2.46723	validation_1-rmse:3.75139
[575]	validation_0-rmse:2.46603	validation_1-rmse:3.75096
[576]	validation_0-rmse:2.46468	validation_1-rmse:3.75064
[577]	validation_0-rmse:2.463	validation_1-rmse:3.75058
[578]	validation_0-rmse:2.46254	validation_1-rmse:3.7519
[579]	validation_0-rmse:2.46032	validation_1-rmse:3.75042
[580]	validation_0-rmse:2.45936	validation_1-rmse:3.75118
[581]	validation_0-rmse:2.45807	validation_1-rmse:3.75015
[582]	validation_0-rmse:2.45732	validation_1-rmse:3.75023
[583]	validation_0-rmse:2.45613	validation_1-rmse:3.74988
[584]	validation_0

...

[1833]	validation_0-rmse:1.53579	validation_1-rmse:3.5891
[1834]	validation_0-rmse:1.53538	validation_1-rmse:3.58878
[1835]	validation_0-rmse:1.53483	validation_1-rmse:3.58791
[1836]	validation_0-rmse:1.53446	validation_1-rmse:3.58804
[1837]	validation_0-rmse:1.53408	validation_1-rmse:3.58814
[1838]	validation_0-rmse:1.53357	validation_1-rmse:3.58838
[1839]	validation_0-rmse:1.53309	validation_1-rmse:3.5883
[1840]	validation_0-rmse:1.53269	validation_1-rmse:3.5888
[1841]	validation_0-rmse:1.53223	validation_1-rmse:3.58876
[1842]	validation_0-rmse:1.53174	validation_1-rmse:3.58805
[1843]	validation_0-rmse:1.53119	validation_1-rmse:3.58772
[1844]	validation_0-rmse:1.53076	validation_1-rmse:3.58791
[1845]	validation_0-rmse:1.53038	validation_1-rmse:3.58762
[1846]	validation_0-rmse:1.5302	validation_1-rmse:3.58769
[1847]	validation_0-rmse:1.52975	validation_1-rmse:3.58729
[1848]	validation_0-rmse:1.52912	validation_1-rmse:3.58714
[1849]	validation_0-rmse:1.52855	validation_1-rmse:3.58665
[

[1973]	validation_0-rmse:1.47053	validation_1-rmse:3.58617
[1974]	validation_0-rmse:1.47012	validation_1-rmse:3.58516
[1975]	validation_0-rmse:1.46965	validation_1-rmse:3.58561
[1976]	validation_0-rmse:1.46911	validation_1-rmse:3.58544
[1977]	validation_0-rmse:1.46899	validation_1-rmse:3.58553
[1978]	validation_0-rmse:1.46887	validation_1-rmse:3.58613
[1979]	validation_0-rmse:1.46832	validation_1-rmse:3.58551
[1980]	validation_0-rmse:1.46783	validation_1-rmse:3.58547
[1981]	validation_0-rmse:1.46756	validation_1-rmse:3.586
[1982]	validation_0-rmse:1.46713	validation_1-rmse:3.58561
[1983]	validation_0-rmse:1.46662	validation_1-rmse:3.58478
[1984]	validation_0-rmse:1.46613	validation_1-rmse:3.58403
[1985]	validation_0-rmse:1.46591	validation_1-rmse:3.58435
[1986]	validation_0-rmse:1.46574	validation_1-rmse:3.58449
[1987]	validation_0-rmse:1.46516	validation_1-rmse:3.58471
[1988]	validation_0-rmse:1.46456	validation_1-rmse:3.58495
[1989]	validation_0-rmse:1.4641	validation_1-rmse:3.5845
[

[2113]	validation_0-rmse:1.41094	validation_1-rmse:3.57809
[2114]	validation_0-rmse:1.41065	validation_1-rmse:3.57834
[2115]	validation_0-rmse:1.41004	validation_1-rmse:3.57774
[2116]	validation_0-rmse:1.40951	validation_1-rmse:3.57751
[2117]	validation_0-rmse:1.40913	validation_1-rmse:3.57761
[2118]	validation_0-rmse:1.40869	validation_1-rmse:3.57726
[2119]	validation_0-rmse:1.40835	validation_1-rmse:3.57746
[2120]	validation_0-rmse:1.408	validation_1-rmse:3.57737
[2121]	validation_0-rmse:1.40751	validation_1-rmse:3.57745
[2122]	validation_0-rmse:1.40709	validation_1-rmse:3.5774
[2123]	validation_0-rmse:1.4068	validation_1-rmse:3.57698
[2124]	validation_0-rmse:1.40625	validation_1-rmse:3.57694
[2125]	validation_0-rmse:1.40582	validation_1-rmse:3.57697
[2126]	validation_0-rmse:1.40527	validation_1-rmse:3.57686
[2127]	validation_0-rmse:1.40488	validation_1-rmse:3.57734
[2128]	validation_0-rmse:1.40434	validation_1-rmse:3.57693
[2129]	validation_0-rmse:1.40401	validation_1-rmse:3.57688
[

[2253]	validation_0-rmse:1.35101	validation_1-rmse:3.566
[2254]	validation_0-rmse:1.35063	validation_1-rmse:3.56591
[2255]	validation_0-rmse:1.35023	validation_1-rmse:3.56592
[2256]	validation_0-rmse:1.34998	validation_1-rmse:3.56611
[2257]	validation_0-rmse:1.34982	validation_1-rmse:3.56601
[2258]	validation_0-rmse:1.34937	validation_1-rmse:3.56504
[2259]	validation_0-rmse:1.34887	validation_1-rmse:3.56502
[2260]	validation_0-rmse:1.34828	validation_1-rmse:3.56448
[2261]	validation_0-rmse:1.34809	validation_1-rmse:3.56444
[2262]	validation_0-rmse:1.34773	validation_1-rmse:3.56528
[2263]	validation_0-rmse:1.34736	validation_1-rmse:3.56542
[2264]	validation_0-rmse:1.34685	validation_1-rmse:3.56516
[2265]	validation_0-rmse:1.3464	validation_1-rmse:3.56522
[2266]	validation_0-rmse:1.34593	validation_1-rmse:3.56486
[2267]	validation_0-rmse:1.34576	validation_1-rmse:3.56481
[2268]	validation_0-rmse:1.34539	validation_1-rmse:3.5648
[2269]	validation_0-rmse:1.34504	validation_1-rmse:3.56482
[

[2393]	validation_0-rmse:1.2985	validation_1-rmse:3.57094
[2394]	validation_0-rmse:1.2983	validation_1-rmse:3.57055
[2395]	validation_0-rmse:1.29793	validation_1-rmse:3.57022
[2396]	validation_0-rmse:1.29747	validation_1-rmse:3.57042
[2397]	validation_0-rmse:1.29702	validation_1-rmse:3.57055
[2398]	validation_0-rmse:1.29659	validation_1-rmse:3.57052
[2399]	validation_0-rmse:1.29619	validation_1-rmse:3.57044
[2400]	validation_0-rmse:1.29587	validation_1-rmse:3.57067
[2401]	validation_0-rmse:1.29536	validation_1-rmse:3.57061
[2402]	validation_0-rmse:1.29508	validation_1-rmse:3.57046
[2403]	validation_0-rmse:1.29471	validation_1-rmse:3.57065
[2404]	validation_0-rmse:1.29435	validation_1-rmse:3.57094
[2405]	validation_0-rmse:1.29401	validation_1-rmse:3.57033
[2406]	validation_0-rmse:1.2936	validation_1-rmse:3.57082
[2407]	validation_0-rmse:1.29307	validation_1-rmse:3.57072
[2408]	validation_0-rmse:1.29262	validation_1-rmse:3.57053
[2409]	validation_0-rmse:1.29215	validation_1-rmse:3.57048


In [213]:
# LightGBM:
model_lgb.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)],eval_metric='rmse',early_stopping_rounds=250)
lgb_train_pred = model_lgb.predict(x_test)
# print(rmsle(y_df, lgb_train_pred))

[1]	training's rmse: 10.2868	valid_1's rmse: 10.2513
Training until validation scores don't improve for 250 rounds
[2]	training's rmse: 10.2764	valid_1's rmse: 10.2419
[3]	training's rmse: 10.2669	valid_1's rmse: 10.2334
[4]	training's rmse: 10.2562	valid_1's rmse: 10.224
[5]	training's rmse: 10.246	valid_1's rmse: 10.2143
[6]	training's rmse: 10.2355	valid_1's rmse: 10.2051
[7]	training's rmse: 10.2241	valid_1's rmse: 10.1941
[8]	training's rmse: 10.2128	valid_1's rmse: 10.1833
[9]	training's rmse: 10.2022	valid_1's rmse: 10.1735
[10]	training's rmse: 10.1912	valid_1's rmse: 10.1634
[11]	training's rmse: 10.1804	valid_1's rmse: 10.1534
[12]	training's rmse: 10.1688	valid_1's rmse: 10.1422
[13]	training's rmse: 10.1582	valid_1's rmse: 10.1326
[14]	training's rmse: 10.1467	valid_1's rmse: 10.1216
[15]	training's rmse: 10.1362	valid_1's rmse: 10.1119
[16]	training's rmse: 10.1253	valid_1's rmse: 10.102
[17]	training's rmse: 10.115	valid_1's rmse: 10.0926
[18]	training's rmse: 10.1046	val

[215]	training's rmse: 8.1265	valid_1's rmse: 8.24159
[216]	training's rmse: 8.11731	valid_1's rmse: 8.23288
[217]	training's rmse: 8.10837	valid_1's rmse: 8.22475
[218]	training's rmse: 8.09948	valid_1's rmse: 8.21622
[219]	training's rmse: 8.0905	valid_1's rmse: 8.20744
[220]	training's rmse: 8.08103	valid_1's rmse: 8.19913
[221]	training's rmse: 8.072	valid_1's rmse: 8.19036
[222]	training's rmse: 8.06296	valid_1's rmse: 8.18187
[223]	training's rmse: 8.05398	valid_1's rmse: 8.17389
[224]	training's rmse: 8.04481	valid_1's rmse: 8.16511
[225]	training's rmse: 8.03552	valid_1's rmse: 8.15641
[226]	training's rmse: 8.02747	valid_1's rmse: 8.14905
[227]	training's rmse: 8.01817	valid_1's rmse: 8.14014
[228]	training's rmse: 8.00845	valid_1's rmse: 8.13091
[229]	training's rmse: 7.99968	valid_1's rmse: 8.12238
[230]	training's rmse: 7.99156	valid_1's rmse: 8.11487
[231]	training's rmse: 7.98254	valid_1's rmse: 8.10643
[232]	training's rmse: 7.97358	valid_1's rmse: 8.09864
[233]	training

[401]	training's rmse: 6.61362	valid_1's rmse: 6.84335
[402]	training's rmse: 6.6064	valid_1's rmse: 6.83684
[403]	training's rmse: 6.59937	valid_1's rmse: 6.83029
[404]	training's rmse: 6.59197	valid_1's rmse: 6.82315
[405]	training's rmse: 6.58445	valid_1's rmse: 6.81617
[406]	training's rmse: 6.57736	valid_1's rmse: 6.80968
[407]	training's rmse: 6.56996	valid_1's rmse: 6.80322
[408]	training's rmse: 6.56268	valid_1's rmse: 6.79624
[409]	training's rmse: 6.55535	valid_1's rmse: 6.78923
[410]	training's rmse: 6.54768	valid_1's rmse: 6.78206
[411]	training's rmse: 6.54047	valid_1's rmse: 6.77573
[412]	training's rmse: 6.53363	valid_1's rmse: 6.76959
[413]	training's rmse: 6.52633	valid_1's rmse: 6.76323
[414]	training's rmse: 6.51989	valid_1's rmse: 6.7572
[415]	training's rmse: 6.51239	valid_1's rmse: 6.74993
[416]	training's rmse: 6.5055	valid_1's rmse: 6.74279
[417]	training's rmse: 6.49851	valid_1's rmse: 6.73598
[418]	training's rmse: 6.4915	valid_1's rmse: 6.72948
[419]	training

[626]	training's rmse: 5.24428	valid_1's rmse: 5.59712
[627]	training's rmse: 5.23898	valid_1's rmse: 5.59242
[628]	training's rmse: 5.23467	valid_1's rmse: 5.58895
[629]	training's rmse: 5.22982	valid_1's rmse: 5.58481
[630]	training's rmse: 5.22466	valid_1's rmse: 5.57984
[631]	training's rmse: 5.2202	valid_1's rmse: 5.5761
[632]	training's rmse: 5.21561	valid_1's rmse: 5.57267
[633]	training's rmse: 5.21036	valid_1's rmse: 5.56763
[634]	training's rmse: 5.20476	valid_1's rmse: 5.56217
[635]	training's rmse: 5.19972	valid_1's rmse: 5.55764
[636]	training's rmse: 5.19439	valid_1's rmse: 5.55258
[637]	training's rmse: 5.19003	valid_1's rmse: 5.54911
[638]	training's rmse: 5.18498	valid_1's rmse: 5.54473
[639]	training's rmse: 5.18026	valid_1's rmse: 5.54051
[640]	training's rmse: 5.17525	valid_1's rmse: 5.53656
[641]	training's rmse: 5.16945	valid_1's rmse: 5.53101
[642]	training's rmse: 5.16471	valid_1's rmse: 5.52704
[643]	training's rmse: 5.16001	valid_1's rmse: 5.52316
[644]	traini

[828]	training's rmse: 4.37679	valid_1's rmse: 4.8403
[829]	training's rmse: 4.37345	valid_1's rmse: 4.83736
[830]	training's rmse: 4.36951	valid_1's rmse: 4.8339
[831]	training's rmse: 4.36592	valid_1's rmse: 4.8308
[832]	training's rmse: 4.36186	valid_1's rmse: 4.82702
[833]	training's rmse: 4.35851	valid_1's rmse: 4.82372
[834]	training's rmse: 4.35513	valid_1's rmse: 4.82078
[835]	training's rmse: 4.35182	valid_1's rmse: 4.81797
[836]	training's rmse: 4.34761	valid_1's rmse: 4.81411
[837]	training's rmse: 4.3449	valid_1's rmse: 4.81159
[838]	training's rmse: 4.34174	valid_1's rmse: 4.80888
[839]	training's rmse: 4.33877	valid_1's rmse: 4.80648
[840]	training's rmse: 4.3349	valid_1's rmse: 4.80316
[841]	training's rmse: 4.33111	valid_1's rmse: 4.79941
[842]	training's rmse: 4.32751	valid_1's rmse: 4.79675
[843]	training's rmse: 4.32471	valid_1's rmse: 4.79441
[844]	training's rmse: 4.32125	valid_1's rmse: 4.79111
[845]	training's rmse: 4.31773	valid_1's rmse: 4.7882
[846]	training's

...

[12511]	training's rmse: 1.67909	valid_1's rmse: 3.46089
[12512]	training's rmse: 1.67905	valid_1's rmse: 3.46089
[12513]	training's rmse: 1.67897	valid_1's rmse: 3.46091
[12514]	training's rmse: 1.67892	valid_1's rmse: 3.4609
[12515]	training's rmse: 1.67886	valid_1's rmse: 3.46094
[12516]	training's rmse: 1.67876	valid_1's rmse: 3.46095
[12517]	training's rmse: 1.67872	valid_1's rmse: 3.46091
[12518]	training's rmse: 1.67866	valid_1's rmse: 3.46088
[12519]	training's rmse: 1.6786	valid_1's rmse: 3.46088
[12520]	training's rmse: 1.67857	valid_1's rmse: 3.46085
[12521]	training's rmse: 1.67856	valid_1's rmse: 3.46083
[12522]	training's rmse: 1.67854	valid_1's rmse: 3.46084
[12523]	training's rmse: 1.67851	valid_1's rmse: 3.46085
[12524]	training's rmse: 1.6785	valid_1's rmse: 3.46085
[12525]	training's rmse: 1.67849	valid_1's rmse: 3.46086
[12526]	training's rmse: 1.67845	valid_1's rmse: 3.46086
[12527]	training's rmse: 1.67841	valid_1's rmse: 3.46084
[12528]	training's rmse: 1.67832	v

[12746]	training's rmse: 1.6698	valid_1's rmse: 3.4594
[12747]	training's rmse: 1.66979	valid_1's rmse: 3.4594
[12748]	training's rmse: 1.66976	valid_1's rmse: 3.45936
[12749]	training's rmse: 1.66974	valid_1's rmse: 3.45936
[12750]	training's rmse: 1.66972	valid_1's rmse: 3.45936
[12751]	training's rmse: 1.66971	valid_1's rmse: 3.45937
[12752]	training's rmse: 1.66967	valid_1's rmse: 3.45937
[12753]	training's rmse: 1.66967	valid_1's rmse: 3.45937
[12754]	training's rmse: 1.66967	valid_1's rmse: 3.45938
[12755]	training's rmse: 1.66966	valid_1's rmse: 3.45936
[12756]	training's rmse: 1.66966	valid_1's rmse: 3.45936
[12757]	training's rmse: 1.66963	valid_1's rmse: 3.45937
[12758]	training's rmse: 1.66963	valid_1's rmse: 3.45937
[12759]	training's rmse: 1.66962	valid_1's rmse: 3.45938
[12760]	training's rmse: 1.66959	valid_1's rmse: 3.45938
[12761]	training's rmse: 1.66958	valid_1's rmse: 3.45938
[12762]	training's rmse: 1.66958	valid_1's rmse: 3.45938
[12763]	training's rmse: 1.66956	v

[13018]	training's rmse: 1.66176	valid_1's rmse: 3.45881
[13019]	training's rmse: 1.66174	valid_1's rmse: 3.45883
[13020]	training's rmse: 1.66171	valid_1's rmse: 3.45882
[13021]	training's rmse: 1.66167	valid_1's rmse: 3.45887
[13022]	training's rmse: 1.66162	valid_1's rmse: 3.45885
[13023]	training's rmse: 1.66161	valid_1's rmse: 3.45885
[13024]	training's rmse: 1.66155	valid_1's rmse: 3.45886
[13025]	training's rmse: 1.66149	valid_1's rmse: 3.45887
[13026]	training's rmse: 1.66147	valid_1's rmse: 3.45886
[13027]	training's rmse: 1.66146	valid_1's rmse: 3.45886
[13028]	training's rmse: 1.66145	valid_1's rmse: 3.45887
[13029]	training's rmse: 1.66144	valid_1's rmse: 3.45887
[13030]	training's rmse: 1.66142	valid_1's rmse: 3.45887
[13031]	training's rmse: 1.66139	valid_1's rmse: 3.45887
[13032]	training's rmse: 1.66139	valid_1's rmse: 3.45887
[13033]	training's rmse: 1.66139	valid_1's rmse: 3.45887
[13034]	training's rmse: 1.66139	valid_1's rmse: 3.45887
[13035]	training's rmse: 1.6614

[13262]	training's rmse: 1.65227	valid_1's rmse: 3.45767
[13263]	training's rmse: 1.65222	valid_1's rmse: 3.4577
[13264]	training's rmse: 1.65213	valid_1's rmse: 3.45758
[13265]	training's rmse: 1.6521	valid_1's rmse: 3.45758
[13266]	training's rmse: 1.65206	valid_1's rmse: 3.45757
[13267]	training's rmse: 1.65201	valid_1's rmse: 3.45751
[13268]	training's rmse: 1.65196	valid_1's rmse: 3.45748
[13269]	training's rmse: 1.65188	valid_1's rmse: 3.45746
[13270]	training's rmse: 1.65183	valid_1's rmse: 3.45745
[13271]	training's rmse: 1.65174	valid_1's rmse: 3.45742
[13272]	training's rmse: 1.65169	valid_1's rmse: 3.45742
[13273]	training's rmse: 1.65165	valid_1's rmse: 3.45737
[13274]	training's rmse: 1.65162	valid_1's rmse: 3.45735
[13275]	training's rmse: 1.6516	valid_1's rmse: 3.45735
[13276]	training's rmse: 1.6516	valid_1's rmse: 3.45735
[13277]	training's rmse: 1.6516	valid_1's rmse: 3.45735
[13278]	training's rmse: 1.6516	valid_1's rmse: 3.45735
[13279]	training's rmse: 1.6516	valid

[13516]	training's rmse: 1.64267	valid_1's rmse: 3.45572
[13517]	training's rmse: 1.64264	valid_1's rmse: 3.45573
[13518]	training's rmse: 1.64258	valid_1's rmse: 3.45569
[13519]	training's rmse: 1.64255	valid_1's rmse: 3.45567
[13520]	training's rmse: 1.6425	valid_1's rmse: 3.45564
[13521]	training's rmse: 1.64242	valid_1's rmse: 3.45567
[13522]	training's rmse: 1.64242	valid_1's rmse: 3.45567
[13523]	training's rmse: 1.64242	valid_1's rmse: 3.45567
[13524]	training's rmse: 1.64242	valid_1's rmse: 3.45567
[13525]	training's rmse: 1.64241	valid_1's rmse: 3.45568
[13526]	training's rmse: 1.6424	valid_1's rmse: 3.45567
[13527]	training's rmse: 1.6424	valid_1's rmse: 3.45567
[13528]	training's rmse: 1.6424	valid_1's rmse: 3.45567
[13529]	training's rmse: 1.6424	valid_1's rmse: 3.45566
[13530]	training's rmse: 1.6424	valid_1's rmse: 3.45566
[13531]	training's rmse: 1.6424	valid_1's rmse: 3.45566
[13532]	training's rmse: 1.64235	valid_1's rmse: 3.45565
[13533]	training's rmse: 1.64229	valid

In [217]:
# GradientBoostingRegressor:
GBoost.fit(x_train, y_train)
GBoost_train_pred =GBoost.predict(x_test)
# print(rmsle(y_df, lgb_train_pred))

In [218]:
averaged_models = AveragingModels(models = (GBoost, model_xgb, model_lgb))
averaged_models.fit(x_train, y_train)
# train_pred = averaged_models.predict(X_test)
# ave_pred = np.expm1(averaged_models.predict(X_test))
# print(rmsle(y_test, ave_pred))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 3.0659 (0.1326)



In [119]:
# cross validation
# sklm.SCORERS.keys()
Results_Lasso = cross_validate(lasso,x_train,y_train,scoring='r2',cv=5,return_train_score=True)
Results_Enet = cross_validate(ENet,x_train,y_train,scoring="r2",cv=5,return_train_score=True)
Results_KRR = cross_validate(KRR,x_train,y_train,scoring="r2",cv=5,return_train_score=True)

In [120]:
Lasso_test_scores = Results_Lasso['test_score']
Lasso_train_scores = Results_Lasso['train_score']
Enet_test_scores = Results_Enet['test_score']
Enet_train_scores = Results_Enet['train_score']
KRR_test_scores = Results_KRR['test_score']
KRR_train_scores = Results_KRR['train_score']

print(np.mean(Lasso_train_scores))
print(np.mean(Lasso_test_scores))
print(np.mean(Enet_train_scores))
print(np.mean(Enet_test_scores))
print(np.mean(KRR_train_scores))
print(np.mean(KRR_test_scores))

0.7946473948570102
0.7881786057783666
0.7968015376626234
0.7889057168734404
0.8508850034850663
0.8144934385140841


In [121]:
estimator = make_pipeline(RobustScaler(), Lasso())
estimator.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('robustscaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('lasso',
                 Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

### prediction

In [219]:
test=pd.read_csv('test_df_enc.csv')
test.shape

(1013, 69)

In [220]:
test.columns

Index(['Unnamed: 0', 'ward', 'total_households', 'total_individuals', 'dw_00',
       'dw_01', 'dw_02', 'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07', 'dw_08',
       'dw_09', 'dw_10', 'dw_11', 'psa_00', 'psa_01', 'psa_02', 'psa_03',
       'psa_04', 'stv_00', 'stv_01', 'car_00', 'car_01', 'lln_00', 'lln_01',
       'lan_00', 'lan_01', 'lan_02', 'lan_03', 'lan_04', 'lan_05', 'lan_06',
       'lan_07', 'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_14',
       'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04', 'lgt_00', 'pw_00', 'pw_01',
       'pw_02', 'pw_03', 'pw_04', 'pw_05', 'pw_06', 'lat', 'lon', 'NL',
       'ADM1_PCODE', 'ADM1_ID', 'ADM2_PCODE', 'ADM2_ID', 'ADM2_TYPE',
       'ADM3_PCODE', 'ADM3_ID', 'ADM3_TYPE', 'average_household_Size',
       'x_cordinate', 'y_cordinate', 'z_cordinate'],
      dtype='object')

In [221]:
test_values = test.drop(['Unnamed: 0', 'ward'], axis=1)
test_values= test_values.fillna(test_values.mean())
test_values.shape

(1013, 67)

In [222]:
test_values=np.array(test_values)
len(test_values)

1013

In [227]:
L_prediccion=pd.DataFrame(model_xgb.predict(test_values),columns=['target'])
L_prediccion['ward']=test['ward']
L_prediccion['target']= round(L_prediccion['target'],6)
L_prediccion.head()

,target,ward
0,20.943977,21001001: Ward 1
1,14.239714,21001002: Ward 2
2,19.605370,21001003: Ward 3
3,18.316576,21001004: Ward 4
4,24.422220,21001005: Ward 5


In [224]:
# Save and submit!
L_prediccion[['ward', 'target']].to_csv('latest_submission.csv', index=False)

In [ ]:
# A hyperparameter optimization framework https://optuna.org
# https://github.com/optuna/optuna